### Imports

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Project path
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data/')

images_data = pd.read_csv(cwd+"/images_data.csv")
images_data['label'] = images_data['label'].astype(str)

train_data = images_data.loc[images_data['classes']=='train']
img_test = images_data.loc[images_data['classes']=='test']

# random_state保证每次分割都一样
img_train, img_valid = train_test_split(train_data, test_size=0.2,random_state = 0)
print(len(img_train), len(img_valid))
print(len(img_test))

19997 5000
12500


In [3]:
img_train.head()

,classes,label,full_path,width,height,ratio
33982,train,1,/home/workspace/capstone/MachineLearningProjec...,500,473,1.057082
14539,train,1,/home/workspace/capstone/MachineLearningProjec...,175,376,0.465426
30594,train,1,/home/workspace/capstone/MachineLearningProjec...,408,365,1.117808
13049,train,1,/home/workspace/capstone/MachineLearningProjec...,375,499,0.751503
30529,train,1,/home/workspace/capstone/MachineLearningProjec...,270,205,1.317073


In [4]:
from keras.preprocessing.image import ImageDataGenerator

target_image_size = (224, 224)

img_gen = ImageDataGenerator(rescale=1.0/255)
train_img_generator = img_gen.flow_from_dataframe(
                                dataframe=img_train,
                                directory=data_dir+'train',
                                x_col='full_path',
                                y_col='label',
                                target_size=target_image_size,
                                class_mode='binary',
                                batch_size=100
                                )

validation_img_generator = img_gen.flow_from_dataframe(
                                dataframe=img_valid,
                                directory=data_dir+'train',
                                x_col='full_path',
                                y_col='label',
                                target_size=target_image_size,
                                class_mode='binary',
                                batch_size=100)

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [4]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard

def train_func(loss_name="binary_crossentropy", optimizer_name="adam"):
    base_model = ResNet50(input_tensor=Input((224, 224, 3)), weights='imagenet',
                          include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    model.compile(loss=loss_name, optimizer=optimizer_name, metrics=['accuracy'])
    best_model = ModelCheckpoint("resnet_best_{}_{}.h5".format(loss_name, optimizer_name),
                                 monitor='val_acc', verbose=0, save_best_only=True)
    tensor_log = "./resnet_best_{}_{}_tensor_log".format(loss_name, optimizer_name)
    return model, best_model, tensor_log
    

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [19]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [26]:
#tf.config.list_physical_devices('GPU')
tf.test.is_gpu_available()

False

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"  # 使用第几个GPU， 0是第一个
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
hello=tf.constant('hhh')
sess=tf.Session()
print (sess.run(hello))


ModuleNotFoundError: No module named 'tensorflow'

In [27]:
print(os.environ['CUDA_VISIBLE_DEVICES'])

0


In [ ]:
model, best_model, tensor_log = train_func("binary_crossentropy", "adam")
model.fit_generator(
        train_img_generator,
        steps_per_epoch=30,
        epochs=50,
        validation_data=validation_img_generator,
        validation_steps=15,
        callbacks=[best_model, TensorBoard(log_dir=tensor_log)])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
 9/30 [========>.....................] - ETA: 2:51 - loss: 0.7765 - accuracy: 0.5156